In [1]:
# dict.fromkeys([1,2,3,4,5])
keys = ["ADJ","ADV","ADP","CONJ","DET","NOUN","NUM","PRON","PRT","VERB","X","."]
keys = [s.lower() for s in keys]
s = {key: {key2:0 for key2 in keys} for key in keys}
# for key in s:
#     print(key)
r = {key: {} for key in keys}
r['adj']['one'] = 0
r


{'adj': {'one': 0},
 'adv': {},
 'adp': {},
 'conj': {},
 'det': {},
 'noun': {},
 'num': {},
 'pron': {},
 'prt': {},
 'verb': {},
 'x': {},
 '.': {}}

In [16]:
# !dir
def read_data(fname):
    exemplars = []
    file = open(fname, 'r');
    for line in file:
        data = tuple([w.lower() for w in line.split()])
        exemplars += [ (data[0::2], data[1::2]), ]

    return exemplars
import sys
# from label import read_data
aaa = read_data('bc.test.tiny')
# for spair in aaa:
#     print(spair[1][0])
#     for i in range(0,len(spair[0])):
#         print(spair[0][i], spair[1][i])

# This train calcs the initial probabilities,ie how likely a sentence begins w/ a certain pos
# as well as the transition probs, eg prob of going from noun to a verb
# TODO:  word dict; how likely a word is a certain pos.
def train(data):
    keys = ["ADJ","ADV","ADP","CONJ","DET","NOUN","NUM","PRON","PRT","VERB","X","."]
    keys = [s.lower() for s in keys]
    init_prob = {key: 0 for key in keys}
    # trans_prob = {key:{key2:0 for key2 in keys} for key in keys}
    trans_prob = {key: {} for key in keys}
    word_prob = {key:{} for key in keys}
    total_sentences = len(data)
    total_words = 0
    for spair in data:
        total_words += len(spair[0])
        # init_prob[spair[1][0]] += 1./total_sentences
        init_prob[spair[1][0]] += 1
        prev_pos = None
        for i in range(0,len(spair[0])):
            current_word = spair[0][i]
            current_pos = spair[1][i]
            try:
                word_prob[current_word][current_pos] += 1
            except:
                # word_prob[current_pos][current_word] = 1
                try:
                    # word_prob[current_word] = {}
                    word_prob[current_word][current_pos] = 1
                except:
                    word_prob[current_word] ={}
                    word_prob[current_word][current_pos] = 1
                try:
                    word_prob[current_word][current_pos] += 1
                except:
                    word_prob[current_word][current_pos] = {}
                # word_prob[current_word][current_pos] = 1
            # Build transition probabilities
            if prev_pos!= None:
                try:
                    trans_prob[prev_pos][current_pos] += 1
                except:
                    trans_prob[prev_pos][current_pos] = 1

            prev_pos = current_pos
    # for pos in trans_prob:
    #     # print(pos)
    #     denom = sum(trans_prob[pos].values())
    #     # print(denom)
    #     for t in trans_prob[pos]:
    #         # print(t,trans_prob[pos][t])
    #     #     # print(t)
    #         if denom != 0:
    #             trans_prob[pos][t] = trans_prob[pos][t]/denom
    return word_prob,init_prob,trans_prob
b = train(aaa)
# sum(b['adj'].values())
b[0]

{'adj': {},
 'adv': {},
 'adp': {},
 'conj': {},
 'det': {},
 'noun': {},
 'num': {},
 'pron': {},
 'prt': {},
 'verb': {},
 'x': {},
 '.': {'.': 3},
 'poet': {'noun': 2},
 'twisted': {'verb': 2},
 'again': {'adv': 2},
 'and': {'conj': 2},
 "nick's": {'noun': 2},
 'knuckles': {'noun': 2},
 'scraped': {'verb': 2},
 'on': {'adp': 2},
 'the': {'det': 5},
 'air': {'noun': 3},
 'tank': {'noun': 2},
 ',': {'.': 4},
 'ripping': {'verb': 2},
 'off': {'prt': 2},
 'skin': {'noun': 2},
 'desperately': {'adv': 2},
 'nick': {'noun': 2},
 'flashed': {'verb': 2},
 'one': {'num': 2},
 'hand': {'noun': 2},
 'up': {'prt': 2},
 'catching': {'verb': 2},
 "poet's": {'noun': 2},
 'neck': {'noun': 2},
 'in': {'adp': 2},
 'bend': {'noun': 2},
 'of': {'adp': 2},
 'his': {'det': 2},
 'elbow': {'noun': 2},
 'hose': {'noun': 2},
 'was': {'verb': 2},
 'free': {'adj': 2},
 '!': {'.': 3}}

In [45]:
aaa = read_data('bc.test.tiny')

In [219]:
from math import log
def get_word_prob(data):
    keys = ["ADJ","ADV","ADP","CONJ","DET","NOUN","NUM","PRON","PRT","VERB","X","."]
    keys = [s.lower() for s in keys]
    print(keys)
    word_dict = {}
    # trans_prob = {key: {} for key in keys}   
    trans_prob = {key:{key2:0 for key2 in keys} for key in keys}

    init_prob = {key: 0 for key in keys}
    total_word = 0
    total_sentences=  len(data)
    for spair in data:
        prev_pos = None 
        total_word += len(spair[0])
        # for each new sentence, check which pos is used at the beginning of the sentence.
        init_prob[spair[1][0]] += 1
        for i in range(0, len(spair[0])):
            word = spair[0][i]
            pos = spair[1][i]
            update_word_prob(word,pos,word_dict)
            prev_pos,trans_prob= update_trans_prob(pos, prev_pos, trans_prob)
    word_dict = finalize_word_dict(word_dict, total_word)
    total_trans = total_word - total_sentences
    trans_prob = finalize_trans_dict(trans_prob, total_trans)
    finalize_init_dict(init_prob, total_sentences)
    return word_dict, trans_prob, init_prob
        # print('\n next sentence: \n')

# transes
# words
# s  = 0
# for t in transes:
#     for m, val in transes[t].items():
#         # print(val)
#         s += val
# s

In [220]:
def update_trans_prob(current_pos, prev_pos, trans_prob):
    # print(prev_pos)
    if prev_pos!= None:
        # print('done')
        try:
            trans_prob[prev_pos][current_pos] += 1
        except:
            trans_prob[prev_pos][current_pos] = 1

    prev_pos = current_pos
    # print(prev_pos)
    return prev_pos, trans_prob
def finalize_trans_dict(trans_prob, total_trans):
    for pos, trans in trans_prob.items():
        for value in trans:
            if trans[value] != 0:
                trans[value] = trans[value] / total_trans
                trans[value] = log(trans[value])
            else:
                trans[value] = -100 
    return trans_prob

def update_word_prob(word,pos,word_dict):
    try:
        word_dict[word][pos] += 1
    except:
        try:
            word_dict[word][pos] = 1
        except:
            word_dict[word] = {}
            word_dict[word][pos] = 1
    return word_dict
def finalize_word_dict(word_dict,total_word):
    
    keys = ["ADJ","ADV","ADP","CONJ","DET","NOUN","NUM","PRON","PRT","VERB","X","."]
    keys = [s.lower() for s in keys]
    # print(keys)
    for word, pos_dict in word_dict.items():
        for key in keys:
            try:
                pos_dict[key] = pos_dict[key] / total_word
                
                pos_dict[key] = log(pos_dict[key])
            except:
                pos_dict[key] = -100
            # print(pos[thing])
            
    return word_dict
def finalize_init_dict(init_dict, total_sentences):
    for pos in init_dict:
        if init_dict[pos] != 0:
            init_dict[pos] = init_dict[pos] / total_sentences
            init_dict[pos] = log(init_dict[pos]) 
        else:
            init_dict[pos] = -100
    return init_dict
# finalize_word_dict(words,10)

In [228]:
v = get_word_prob(aaa)
# v['hand']
words,transes, inits = v
# transes
# inits
a_dictionary = words['poet']
max(words['poet'],key = words['poet'].get)
# max_key = max(a_dictionary, key=a_dictionary.get)
# print(max_key)
# a_dictionary
# a_dictionary[maxkey]
# max(words['poet'])
# transes
# # test_sentence = 
# # aaa[2][0]
test_sentence = ('the', 'air', 'hose', 'was', 'free', '!')
words
# for word in test_sentence:
#     most_likely_simple = max(words[word], key= words[word].get)
#     print(most_likely_simple)

['adj', 'adv', 'adp', 'conj', 'det', 'noun', 'num', 'pron', 'prt', 'verb', 'x', '.']


{'poet': {'noun': -3.7376696182833684,
  'adj': -100,
  'adv': -100,
  'adp': -100,
  'conj': -100,
  'det': -100,
  'num': -100,
  'pron': -100,
  'prt': -100,
  'verb': -100,
  'x': -100,
  '.': -100},
 'twisted': {'verb': -3.7376696182833684,
  'adj': -100,
  'adv': -100,
  'adp': -100,
  'conj': -100,
  'det': -100,
  'noun': -100,
  'num': -100,
  'pron': -100,
  'prt': -100,
  'x': -100,
  '.': -100},
 'again': {'adv': -3.7376696182833684,
  'adj': -100,
  'adp': -100,
  'conj': -100,
  'det': -100,
  'noun': -100,
  'num': -100,
  'pron': -100,
  'prt': -100,
  'verb': -100,
  'x': -100,
  '.': -100},
 'and': {'conj': -3.7376696182833684,
  'adj': -100,
  'adv': -100,
  'adp': -100,
  'det': -100,
  'noun': -100,
  'num': -100,
  'pron': -100,
  'prt': -100,
  'verb': -100,
  'x': -100,
  '.': -100},
 "nick's": {'noun': -3.7376696182833684,
  'adj': -100,
  'adv': -100,
  'adp': -100,
  'conj': -100,
  'det': -100,
  'num': -100,
  'pron': -100,
  'prt': -100,
  'verb': -100,
  

In [225]:
log(0.999)+log(0.513)+log(0.839)+log(0.392)+log(0.999)+log(0.999)

-1.7825189465187237

In [64]:
def get_trans_prob(data):
    keys = ["ADJ","ADV","ADP","CONJ","DET","NOUN","NUM","PRON","PRT","VERB","X","."]
    keys = [s.lower() for s in keys]
    trans_prob = {key: {} for key in keys}
    for spair in data:
    return trans_prob
get_trans_prob(aaa)


{'adj': {},
 'adv': {},
 'adp': {},
 'conj': {},
 'det': {},
 'noun': {},
 'num': {},
 'pron': {},
 'prt': {},
 'verb': {},
 'x': {},
 '.': {}}

NameError: name 'word_dict' is not defined

In [12]:
len(b)
# print(b[0]['do'])
# print(b[1])
# print(b[2]['adj'])
trained_word,trained_init,trained_trans = b
maxkey = ''
maxval = 0
# print(trained_word)
for key,value in trained_word['was'].items():
    # print(key)
    for k,v in trained_init.items():
        for kay, vee in trained_trans[k].items():
            # print(value,v,vee)
            if maxval < value*v*vee:
                maxval = value*v*vee
                maxkey = key
print(maxkey,maxval)
# def viterbi(words):
# trained_word
# for item in b['do']:
#     print(item)

verb 10


In [8]:
trained_word

 {'verb': 137, 'noun': 523},
 'officials': {'noun': 61},
 'pass': {'verb': 55, 'noun': 17},
 'legislation': {'noun': 47},
 'permit': {'verb': 64, 'noun': 8},
 'establishment': {'noun': 50},
 'fair': {'adj': 58, 'noun': 8, 'adv': 2},
 'equitable': {'adj': 11},
 'pension': {'noun': 12},
 'plan': {'noun': 164, 'verb': 30},
 'employes': {'noun': 18},
 'praised': {'verb': 13},
 'operation': {'noun': 112},
 'police': {'noun': 117},
 'tax': {'noun': 194, 'verb': 6},
 "commissioner's": {'noun': 2},
 'office': {'noun': 199},
 'bellwood': {'noun': 2},
 'alpharetta': {'noun': 2},
 'prison': {'noun': 35},
 'farms': {'noun': 15},
 'grady': {'noun': 6},
 'hospital': {'noun': 100},
 'health': {'noun': 101},
 'mayor': {'noun': 36},
 'william': {'noun': 125},
 'b.': {'noun': 77},
 'hartsfield': {'noun': 6},
 'filed': {'verb': 32},
 'suit': {'noun': 29, 'verb': 9},
 'divorce': {'noun': 17, 'verb': 6},
 'his': {'det': 4952, 'pron': 20, 'x': 3},
 'wife': {'noun': 164},
 'pearl': {'noun': 9},
 'williams': 

In [4]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        try:          
            V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
        except:
            pass
        

In [5]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        try:          
            V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
        except:
            pass
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            try:
                max_tr_prob = V[t - 1][states[0]]["prob"] * trans_p[states[0]][st]
                prev_st_selected = states[0]
                for prev_st in states[1:]:
                    tr_prob = V[t - 1][prev_st]["prob"] * trans_p[prev_st][st]
                    if tr_prob > max_tr_prob:
                        max_tr_prob = tr_prob
                        prev_st_selected = prev_st

                max_prob = max_tr_prob * emit_p[st][obs[t]]
                V[t][st] = {"prob": max_prob, "prev": prev_st_selected}
            except:
                pass

    for line in dptable(V):
        print(line)

    opt = []
    max_prob = 0.0
    previous = None
    best_st = 'adj'
    # Get most probable state and its backtrack

    for st, data in V[-1].items():
        
        best_st = st
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    print ('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s' % max_prob)

def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        try:
            yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)
        except:
            pass


wordlist = ["ADJ","ADV","ADP","CONJ","DET","NOUN","NUM","PRON","PRT","VERB","X","."]
wordlist = [s.lower() for s in wordlist]
print(trained_init['noun'])
viterbi(['this','is','a','test'],wordlist, trained_init,trained_trans,trained_word )

6539
           0            1            2            3


KeyError: 'adj'

In [ ]:
c = {}
# c['one'] = 1
try:
    c['one'].update({'a':1})
except:
    c['one'] = {}
    c['one'].update({'a':1})
c